In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

path = "Date_Fruit_Datasets.xlsx"
df = pd.read_excel(io=path,sheet_name="Date_Fruit_Datasets")

X, y = df.drop(['Class'],axis=1), df['Class']

# duomenys antrai užduočiai
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75,stratify=y, random_state=0)
X.shape,y.shape

# duomenys pirmai užduočiai
X1, X_train1, X_test1, y1, y_train1, y_test1 = X.iloc[:,1:],X_train.iloc[:,1:],X_test.iloc[:,1:],X.iloc[:,0],X_train.iloc[:,0], X_test.iloc[:,0]

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# 1

In [2]:
from sklearn.ensemble import RandomForestRegressor

## a

In [3]:
from sklearn.feature_selection import f_regression
aimportance = pd.DataFrame.from_dict(
    {
        'f-statistic':f_regression(X_train1, y_train1)[0],
        'col' : X_train1.columns,
    }
).sort_values(by = 'f-statistic', ascending=False).iloc[:15].reset_index(drop = True)
aimportance

,f-statistic,col
0,313072.971736,CONVEX_AREA
1,46309.734909,EQDIASQ
2,6912.783610,PERIMETER
3,6579.097859,MINOR_AXIS
4,2123.690977,MAJOR_AXIS
5,1917.124935,SHAPEFACTOR_2
6,238.957094,EntropyRB
7,149.401720,KurtosisRB
8,149.054148,SkewRB
9,128.956544,SkewRG


In [4]:
areg = RandomForestRegressor(random_state=0).fit(X_train1[aimportance.col], y_train1)

## b

In [5]:
from sklearn.feature_selection import mutual_info_regression
bimportance = pd.DataFrame.from_dict(
    {
        'MI':mutual_info_regression(X_train1, y_train1),
        'col' : X_train1.columns,
    }
).sort_values(by = 'MI', ascending=False).iloc[:15].reset_index(drop = True)
bimportance

,MI,col
0,5.095102,EQDIASQ
1,3.515899,CONVEX_AREA
2,1.837679,PERIMETER
3,1.424844,MINOR_AXIS
4,1.199323,MAJOR_AXIS
5,1.147255,SHAPEFACTOR_1
6,1.106814,SHAPEFACTOR_2
7,0.406614,EntropyRR
8,0.358638,ECCENTRICITY
9,0.345830,COMPACTNESS


In [6]:
breg = RandomForestRegressor(random_state=0).fit(X_train1[bimportance.col], y_train1)

## c

In [7]:
apred = areg.predict(X_test1[aimportance.col])
bpred = breg.predict(X_test1[bimportance.col])

In [8]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [9]:
print("a) R^2:", r2_score(y_test1,apred), " MAPE: ", mean_absolute_percentage_error(y_test1,apred))
print("b) R^2:", r2_score(y_test1,bpred), " MAPE: ", mean_absolute_percentage_error(y_test1,bpred))

a) R^2: 0.9996247015827058  MAPE:  0.004341334223933075
b) R^2: 0.9997426763537302  MAPE:  0.004191853543546487


# d

In [10]:
print("a) kintamieji")
aimportance.col.to_list()

a) kintamieji


['CONVEX_AREA',
 'EQDIASQ',
 'PERIMETER',
 'MINOR_AXIS',
 'MAJOR_AXIS',
 'SHAPEFACTOR_2',
 'EntropyRB',
 'KurtosisRB',
 'SkewRB',
 'SkewRG',
 'EntropyRG',
 'ALLdaub4RG',
 'MeanRG',
 'ALLdaub4RB',
 'MeanRB']

In [11]:
print("b) kintamieji")
bimportance.col.to_list()

b) kintamieji


['EQDIASQ',
 'CONVEX_AREA',
 'PERIMETER',
 'MINOR_AXIS',
 'MAJOR_AXIS',
 'SHAPEFACTOR_1',
 'SHAPEFACTOR_2',
 'EntropyRR',
 'ECCENTRICITY',
 'COMPACTNESS',
 'SHAPEFACTOR_3',
 'EntropyRG',
 'ROUNDNESS',
 'EntropyRB',
 'ALLdaub4RG']

# 2

In [12]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.preprocessing import MaxAbsScaler

## a

In [13]:
apipe = Pipeline(
    [
        ("minmax", MinMaxScaler(feature_range = (-1, 1))), 
        ("pca",  PCA()),
        ("LR", LogisticRegression(random_state=0, max_iter = 1000000)),
    ]
)

agrid = dict(
    pca__n_components =  [2, 4, 6, 8, 16, 32], 
    LR__C = [0.01, 0.1, 1, 10, 100],
)

asearch = GridSearchCV(apipe, param_grid=agrid, verbose=1).fit(X_train, y_train)
print("Geriausio modelio tikslumas testinėje aibėje: %.3f" % asearch.score(X_test, y_test))
print(classification_report(asearch.predict(X_test), y_test))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Geriausio modelio tikslumas testinėje aibėje: 0.916
              precision    recall  f1-score   support

       BERHI       0.88      0.82      0.85        17
      DEGLET       0.80      0.80      0.80        25
       DOKOL       0.92      0.96      0.94        49
       IRAQI       0.83      1.00      0.91        15
      ROTANA       1.00      0.91      0.95        46
      SAFAVI       1.00      0.98      0.99        51
       SOGAY       0.78      0.82      0.80        22

    accuracy                           0.92       225
   macro avg       0.89      0.90      0.89       225
weighted avg       0.92      0.92      0.92       225



In [14]:
# Matome kad 78% SOGAY klasės spėjimų buvo teisingi, galimai per dažnai spėjama ši klasė
# Atspėjome visus IRAQI klasės datules ir tik 80% DEGLET klasės
# Kitur rezultatai geri

## b

In [15]:
bpipe = Pipeline(
    [
        ("minmax", MinMaxScaler(feature_range = (0, 1))), 
        ("lle",   LocallyLinearEmbedding()),
        ("LR", LogisticRegression(random_state=0, max_iter = 1000000)),
    ]
)

bgrid = dict(
    lle__n_components =  [2, 8, 32, 34], 
    LR__C = [0.01, 0.1, 1, 10, 100],
)

bsearch = GridSearchCV(bpipe, param_grid=bgrid, verbose=1).fit(X_train, y_train)
print("Geriausio modelio tikslumas testinėje aibėje: %.3f" % bsearch.score(X_test, y_test))
print(classification_report(bsearch.predict(X_test), y_test))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Geriausio modelio tikslumas testinėje aibėje: 0.871
              precision    recall  f1-score   support

       BERHI       0.69      0.61      0.65        18
      DEGLET       0.64      0.76      0.70        21
       DOKOL       0.94      0.91      0.92        53
       IRAQI       0.72      0.87      0.79        15
      ROTANA       1.00      0.98      0.99        43
      SAFAVI       0.96      0.98      0.97        49
       SOGAY       0.78      0.69      0.73        26

    accuracy                           0.87       225
   macro avg       0.82      0.83      0.82       225
weighted avg       0.88      0.87      0.87       225



In [16]:
# Sąlyginai prastai spėjama BERHI klasė
# Tik 61% jų buvo identifikuota teisingai ir tik 69% spėjimų kad tai ši klasį buvo teisingi
# Taip pat buvo sunkumų su DEGLET klase

## c

In [17]:
cpipe = Pipeline(
    [
        ("standard", StandardScaler()), 
        ("lda",   LinearDiscriminantAnalysis()),
        ("LR", LogisticRegression(random_state=0, max_iter = 1000000)),
    ]
)

cgrid = dict(
    lda__n_components =  [1, 2, 3, 4, 5, 6], 
    LR__C = [0.01, 0.1, 1, 10, 100],
)

csearch = GridSearchCV(cpipe, param_grid=cgrid, verbose=1).fit(X_train, y_train)
print("Geriausio modelio tikslumas testinėje aibėje: %.3f" % csearch.score(X_test, y_test))
print(classification_report(csearch.predict(X_test), y_test))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Geriausio modelio tikslumas testinėje aibėje: 0.933
              precision    recall  f1-score   support

       BERHI       0.94      0.79      0.86        19
      DEGLET       0.84      0.84      0.84        25
       DOKOL       0.92      0.96      0.94        49
       IRAQI       0.83      0.94      0.88        16
      ROTANA       1.00      0.91      0.95        46
      SAFAVI       0.98      1.00      0.99        49
       SOGAY       0.91      1.00      0.95        21

    accuracy                           0.93       225
   macro avg       0.92      0.92      0.92       225
weighted avg       0.94      0.93      0.93       225



In [18]:
# Visos klasės spėjamos labai gerai

### b modelis be skales keitimo

In [19]:
b2pipe = Pipeline(
    [
        ("lle",   LocallyLinearEmbedding()),
        ("LR", LogisticRegression(random_state=0, max_iter = 1000000)),
    ]
)

b2grid = dict(
    lle__n_components =  [2, 8, 32, 34], 
    LR__C = [0.01, 0.1, 1, 10, 100],
)

b2search = GridSearchCV(b2pipe, param_grid=b2grid, verbose=1).fit(X_train, y_train)
print("Geriausio modelio tikslumas testinėje aibėje: %.3f" % b2search.score(X_test, y_test))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Geriausio modelio tikslumas testinėje aibėje: 0.702


In [20]:
# Rezultatas suprastejo mazdaug per 17%

### a modelis su maksimumo normavimu

In [21]:
a2pipe = Pipeline(
    [
        ("maxabs", MaxAbsScaler()), 
        ("pca",  PCA()),
        ("LR", LogisticRegression(random_state=0, max_iter = 1000000)),
    ]
)

a2grid = dict(
    pca__n_components =  [2, 4, 6, 8, 16, 32], 
    LR__C = [0.01, 0.1, 1, 10, 100],
)

a2search = GridSearchCV(a2pipe, param_grid=a2grid, verbose=1).fit(X_train, y_train)
print("Geriausio modelio tikslumas testinėje aibėje: %.3f" % a2search.score(X_test, y_test))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Geriausio modelio tikslumas testinėje aibėje: 0.916


In [22]:
# Rezultatas nepasikeite